# Notebook "4_Besoin_3_Application_lissage"


## Description

Ce notebook se focalise sur la récupération des données stocké en DB de l'application du lissage

## Structure du notebook

Ce notebook est construit de plusieurs parties :
- I. Import des bibliothèques
- II. Chargement des données et des profils
- III. Affichage des résultats
- IV. Comparaison avec le signal reconstruit

## Prérequis

### Source de données

Pour que le notebook fonctionne, il est nécessaire que le fichier project_config.yml soit situé au même niveau que ce notebook : ce fichier permet la connexion à la source de données.

### Installation des prérequis techniques

Le notebook fonctionne sur Python 3.7 avec les bibliothèques suivantes :
- pandas
- psycopg2
- sqlachemy
- sklearn
- numpy
- yaml
- matplotlib

# I - Import des bibliothèques

In [1]:
# handling postgres database
import psycopg2
import pandas.io.sql as sqlio
import pandas as pd 
from sqlalchemy import create_engine
from io import StringIO
from sklearn.linear_model import Lasso
from sklearn import metrics
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import yaml
from IPython.display import clear_output

# II - Chargement des données et des profils

## II.a Recherche des identifiants de connexion

In [2]:
with open(r'.\project_config.yml') as file:
    dbInfo = yaml.load(file, Loader=yaml.FullLoader)
    HOSTNAME = dbInfo["project-database"]["hostname"]
    DATABASE = dbInfo["project-database"]["name"]
    USER = dbInfo["project-database"]["user"]
    PASSWORD = dbInfo["project-database"]["password"]
    PORT = "5432"

## II.b Recherche des profils dans la DB

In [3]:
query = "SELECT * FROM public.profiles"
connection = psycopg2.connect(user = USER, password = PASSWORD, host = HOSTNAME, port = PORT, database = DATABASE)
df = sqlio.read_sql_query(query, connection)
connection.close()

In [10]:
Profiles = pd.unique(df['profile'])

array(['BBOA', 'HOA', 'LO-OOA', 'MO-OOA'], dtype=object)

## II.c Récupération des données

In [5]:
## SELECTION DE LA DATE
start_date = '2011-11-01'
end_date = '2011-11-02'

# start_date = '2017-01-01'
# end_date = '2019-01-01'

In [6]:
query = f"""
    SELECT *
    FROM public.regressor_results
    WHERE model = 'LASSO_x3' AND date >= '{start_date}' AND date <= '{end_date}'
    ORDER BY date
"""
connection = psycopg2.connect(user = USER, password = PASSWORD, host = HOSTNAME, port = PORT, database = DATABASE)
df_result = sqlio.read_sql_query(query, connection)
connection.close()

In [7]:
list_date = pd.unique(df_result["date"])
df_final = pd.DataFrame(data=[], columns=["date"])
df_final['date'] = list_date

In [11]:
for pro in Profiles:
    result_inter = []
    aux = df_result[['date', 'contribution']][df_result['profile'] == pro]
    aux = aux.sort_values(by=["date"])
    df_final[pro] = aux['contribution'].values


In [15]:
df_final[Profiles].values

array([[5.98633503, 4.63009675, 5.67239699, 0.22554461],
       [5.48126882, 4.20238788, 4.94589835, 0.56828839],
       [5.05678201, 3.76038995, 4.21442789, 0.88251284],
       [4.57249579, 3.26561419, 5.33422826, 0.24344081],
       [4.30594027, 3.0591166 , 4.58301912, 0.72028385],
       [2.74517168, 2.65550475, 4.59021283, 1.27171596],
       [1.98664804, 2.48302608, 3.83363703, 2.0182603 ],
       [1.51101152, 2.90954654, 4.74918996, 2.05245814],
       [3.13304398, 3.57359757, 4.56218703, 1.97843558],
       [4.54379841, 4.1754984 , 4.52053704, 1.31009992],
       [5.12384859, 3.79599705, 3.4626779 , 1.16335253],
       [6.02206898, 3.77491765, 1.7601888 , 1.66550783],
       [6.08541888, 3.45933101, 0.45491638, 2.37024274],
       [6.1223858 , 3.42577751, 0.40033787, 2.46134295],
       [6.28776425, 3.3740633 , 0.30524692, 2.54884377],
       [7.33785584, 3.80320468, 0.        , 2.63357436],
       [8.79723513, 4.12466878, 0.        , 2.29670955],
       [9.55193875, 4.41746937,

In [17]:
df_pro = pd.DataFrame(data=[], columns=["mass"])
df_pro['mass'] = pd.unique(df['mass'])

In [19]:
for pro in Profiles:
    result_inter = []
    aux = df[['mass', 'value']][df['profile'] == pro]
    aux = aux.sort_values(by=["mass"])
    df_pro[pro] = aux['value'].values

# IV. Comparaison avec le signal reconstruit

## IV.a Reconstruction du signal

In [27]:
reconstruct = np.dot(df_final[Profiles].values, np.transpose(df_pro[Profiles].values))

In [31]:
reconstruct

array([[0.04255116, 0.563124  , 0.08773093, ..., 0.04399869, 0.05248219,
        0.01975264],
       [0.03924732, 0.50904086, 0.08234089, ..., 0.04016314, 0.04800137,
        0.01809233],
       [0.03594559, 0.45943503, 0.07708697, ..., 0.03628746, 0.0436769 ,
        0.01653763],
       ...,
       [0.03422789, 0.55939978, 0.08531219, ..., 0.05193237, 0.05926176,
        0.02265422],
       [0.03348069, 0.47355758, 0.0770168 , ..., 0.04812594, 0.05251588,
        0.01937157],
       [0.0281852 , 0.40451627, 0.06749536, ..., 0.04278035, 0.04634008,
        0.01712109]])

## IV.b Récupération du signal original

In [32]:
connection = psycopg2.connect(user = USER, password = PASSWORD, host = HOSTNAME, port = PORT, database = DATABASE)
query = f"SELECT * FROM public.data_receptor WHERE date > '{start_date}' AND date <= '{end_date}'"
df_receptor = sqlio.read_sql_query(query, connection)
connection.close()

df_pivot = df_receptor.pivot_table(values="value", columns="mass", index="date")
df_pivot = df_pivot.sort_values("date")
df_array = df_pivot.reset_index().values
df_array = [i[1:] for i in df_array]

## IV.c Comparaison des deux signaux

In [38]:
df_avg_array = []
for i in range(len(df_array) - 2):
    aux = df_array[i] + df_array[i + 1] + df_array[i + 2]
    aux = aux / 3
    df_avg_array.append(aux)

In [139]:
np.sum(np.abs(df_avg_array - reconstruct), axis=1) / len(test_1)

array([0.02239664934437507, 0.0205952823371783, 0.018488787123914944,
       0.017851099186993808, 0.016013936901963753, 0.016800042660327582,
       0.017384941845785443, 0.02203240866505243, 0.022352040052723936,
       0.021026149913748737, 0.01766945487057736, 0.017230205414871677,
       0.01608944489386541, 0.01506018517854866, 0.013697850124412353,
       0.017391727561609904, 0.018702644440032272, 0.019223344071428602,
       0.02061946519934335, 0.02384652851643225, 0.02319897534507463,
       0.01933822925227194], dtype=object)

In [140]:
df_avg_array

[array([0.010517676666666668, 0.5850853333333332, 0.08803346666666667,
        0.5502093333333334, 2.200833333333333, 0.009623223333333333,
        0.06142773333333334, 0.16763266666666665, 0.8282816666666667,
        1.44756, 0.031846366666666674, 0.36334133333333335,
        0.07911746666666666, 0.10329683333333334, 0.8531413333333333,
        0.3362773333333333, 1.1334706666666665, 2.200833333333333,
        0.22720666666666667, 0.013137523333333333, 0.00886435,
        0.08962303333333332, 0.07589776666666666, 0.08299959999999999,
        0.207129, 0.1079865, 0.419354, 0.19638999999999998,
        0.32178766666666664, 0.09482643333333333, 0.08680756666666667,
        0.217363, 0.0416506, 0.026275033333333336, 0.0855809, 0.0743658,
        0.11156473333333335, 0.053971766666666664, 0.20213633333333333,
        0.12939266666666668, 0.31092433333333336, 0.1031379, 0.205469,
        0.0366454, 0.19232733333333332, 0.07513056666666666,
        0.047839400000000004, 0.03490406666666667, 